## Cifar-10 CNN
### Import, init

In [1]:
import keras
import numpy as np
import pickle

keras.backend.set_image_data_format('channels_first')
print('Using data format:', keras.backend.image_data_format())

Using TensorFlow backend.


Using data format: channels_first


### Read data

In [2]:
img_row, img_col, img_cnl = 32, 32, 3
label_names = { }
def read_imgs(filename):
    f = open(filename, 'rb')
    data = pickle.load(f, encoding='bytes')
    f.close()
    imgs = np.array(data[b'data'])
    imgs = imgs.reshape((imgs.shape[0], img_cnl, img_row, img_col))
    labels = np.array(data[b'labels'])
    return imgs, labels
def load_labels(filename):
    f = open(filename, 'rb')
    data = pickle.load(f, encoding='bytes')
    f.close()
    return data[b'label_names']
print('# test: ')
%time imgs, labels = read_imgs('datas/data_batch_1')
print('# data shape:' ,imgs.shape)
print('# label names')
print(load_labels('datas/batches.meta'))

# test: 
Wall time: 47.6 ms
# data shape: (10000, 3, 32, 32)
# label names
[b'airplane', b'automobile', b'bird', b'cat', b'deer', b'dog', b'frog', b'horse', b'ship', b'truck']


read all data from file

In [3]:
# Load datas
train_x, train_y = read_imgs('datas/data_batch_1')
for x in (2, 3, 4, 5):
    fn = 'datas/data_batch_%d' % x
    imgs, labels = read_imgs(fn)
    train_x = np.concatenate((train_x, imgs))
    train_y = np.concatenate((train_y, labels))
test_x, test_y = read_imgs('datas/test_batch')

# Load label names
label_names = load_labels('datas/batches.meta')
label_names = [x.decode('ascii') for x in label_names]

# Categorical
class_cnt = len(label_names)
train_y = keras.utils.to_categorical(train_y, class_cnt)
test_y = keras.utils.to_categorical(test_y, class_cnt)

# Standard
train_x = train_x.astype('float32')
test_x = test_x.astype('float32')
train_x /= 255
test_x /= 255

print('# Train data loaded:', train_x.shape[0])
print('# Test data loaded:', test_x.shape[0])
print('# Label names:')
print(label_names)

# Train data loaded: 50000
# Test data loaded: 10000
# Label names:
['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


## Define network structure
Conv -> Conv -> Pooling -> Conv -> Conv -> Pooling -> Dense

In [ ]:
batch_size = 128
epochs = 300

model = keras.models.Sequential()

model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=train_x.shape[1:]))
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(512, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(class_cnt, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

## Train and evaluate

In [ ]:
model.fit(train_x, train_y,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(test_x, test_y))
model.evaluate(test_x, test_y, verbose=0)

Train on 50000 samples, validate on 10000 samples
Epoch 1/300
50000/50000 [==============================] - 42s - loss: 1.6373 - acc: 0.3964 - val_loss: 1.2822 - val_acc: 0.5332
Epoch 2/300
50000/50000 [==============================] - 40s - loss: 1.2409 - acc: 0.5564 - val_loss: 1.0503 - val_acc: 0.6285
Epoch 3/300
50000/50000 [==============================] - 40s - loss: 1.0757 - acc: 0.6197 - val_loss: 0.9694 - val_acc: 0.6571
Epoch 4/300
50000/50000 [==============================] - 40s - loss: 0.9542 - acc: 0.6647 - val_loss: 0.8585 - val_acc: 0.6989
Epoch 5/300
50000/50000 [==============================] - 40s - loss: 0.8695 - acc: 0.6965 - val_loss: 0.8476 - val_acc: 0.7030
Epoch 6/300
50000/50000 [==============================] - 40s - loss: 0.8022 - acc: 0.7181 - val_loss: 0.7387 - val_acc: 0.7437
Epoch 7/300
50000/50000 [==============================] - 40s - loss: 0.7409 - acc: 0.7433 - val_loss: 0.7120 - val_acc: 0.7491
Epoch 8/300
50000/50000 [======================

50000/50000 [==============================] - 40s - loss: 0.2271 - acc: 0.9210 - val_loss: 0.6650 - val_acc: 0.8125
Epoch 64/300
50000/50000 [==============================] - 40s - loss: 0.2341 - acc: 0.9189 - val_loss: 0.6839 - val_acc: 0.8113
Epoch 65/300
50000/50000 [==============================] - 40s - loss: 0.2225 - acc: 0.9210 - val_loss: 0.6877 - val_acc: 0.8143
Epoch 66/300
50000/50000 [==============================] - 39s - loss: 0.2273 - acc: 0.9209 - val_loss: 0.6955 - val_acc: 0.8089
Epoch 67/300
50000/50000 [==============================] - 40s - loss: 0.2223 - acc: 0.9216 - val_loss: 0.6773 - val_acc: 0.8121
Epoch 68/300
50000/50000 [==============================] - 40s - loss: 0.2248 - acc: 0.9217 - val_loss: 0.6704 - val_acc: 0.8100
Epoch 69/300
50000/50000 [==============================] - 40s - loss: 0.2249 - acc: 0.9201 - val_loss: 0.6884 - val_acc: 0.8070
Epoch 70/300
50000/50000 [==============================] - 40s - loss: 0.2125 - acc: 0.9267 - val_loss

50000/50000 [==============================] - 39s - loss: 0.1751 - acc: 0.9405 - val_loss: 0.7173 - val_acc: 0.8133
Epoch 126/300
50000/50000 [==============================] - 39s - loss: 0.1729 - acc: 0.9398 - val_loss: 0.7185 - val_acc: 0.8144
Epoch 127/300
50000/50000 [==============================] - 40s - loss: 0.1748 - acc: 0.9403 - val_loss: 0.7743 - val_acc: 0.8069
Epoch 128/300
50000/50000 [==============================] - 39s - loss: 0.1765 - acc: 0.9395 - val_loss: 0.7488 - val_acc: 0.8069
Epoch 129/300
50000/50000 [==============================] - 40s - loss: 0.1722 - acc: 0.9411 - val_loss: 0.7502 - val_acc: 0.8111
Epoch 130/300
50000/50000 [==============================] - 40s - loss: 0.1711 - acc: 0.9419 - val_loss: 0.7125 - val_acc: 0.8119
Epoch 131/300
50000/50000 [==============================] - 40s - loss: 0.1671 - acc: 0.9422 - val_loss: 0.7332 - val_acc: 0.8132
Epoch 132/300
50000/50000 [==============================] - 40s - loss: 0.1705 - acc: 0.9413 - v

## Save the model

In [ ]:
model.save('trained_model2.h5')